# Custom Routing Chains

This covers how to implement a custom routing chain. That problem really reduces to how to implement a custom router. This also acts as a design doc of sorts for routers.

## Terminology

Before going through any code, let's align on some terminology.
- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- Tool Input: The input string to a tool.
- Observation: The output from calling a tool on a particular input.
- Router: The object responsible for deciding which tools to call and when. Exposes a `route` method, which takes in a string and returns a Router Output.
- Router Output: The object returned from calling `Router.route` on a string. Consists of:
    - The tool to use
    - The input to that tool
    - A log of the router's thinking.
- Routing Chain: A chain which is made up of a router and suite of tools. When passed a string, the Routing Chain will iterative call tools as needed until it arrives at a Final Answer.
- Final Answer: The final output of a Routing Chain.

## Router
A central piece of this chain is the router. The router is responsible for taking user input and deciding which tools, if any, to use. Although it doesn't necessarily have to be backed by a language model (LLM), for pretty much all current use cases it is. LLMs make great routers because they are really good at understanding human intent, which makes them perfect for choosing which tools to use (and for interpreting the output of those tools).

Below is the interface we expect routers to expose, along with the RouterOutput definition.

```python

class RouterOutput(NamedTuple):
    """Output of a router."""

    tool: str
    tool_input: str
    log: str
        

class Router(ABC):
    """Chain responsible for deciding the action to take."""

    @abstractmethod
    def route(self, text: str) -> RouterOutput:
        """Given input, decided how to route it.

        Args:
            text: input string

        Returns:
            RouterOutput specifying what tool to use.
        """

    @property
    @abstractmethod
    def observation_prefix(self) -> str:
        """Prefix to append the observation with before calling the router again."""

    @property
    @abstractmethod
    def router_prefix(self) -> str:
        """Prefix to prepend the router call with."""

    @property
    def finish_tool_name(self) -> str:
        """Name of the tool to use to finish the chain."""
        return "Final Answer"

    @property
    def starter_string(self) -> str:
        """Put this string after user input but before first router call."""
        return "\n"
```

In order to understand why the router interface is what it is, let's take a look at how it is used in the RoutingChain class:

```python
def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
    # Construct a mapping of tool name to tool for easy lookup
    name_to_tool_map = {tc.tool_name: tc.tool for tc in self.tool_configs}
    # Construct the initial string to pass into the router. This is made up
    # of the user input, the special starter string, and then the router prefix.
    # The starter string is a special string that may be used by a router to
    # immediately follow the user input. The router prefix is a string that
    # prompts the router to start routing.
    starter_string = (
        inputs[self.input_key]
        + self.router.starter_string
        + self.router.router_prefix
    )
    # We use the ChainedInput class to iteratively add to the input over time.
    chained_input = ChainedInput(starter_string, verbose=self.verbose)
    # We construct a mapping from each tool to a color, used for logging.
    color_mapping = get_color_mapping(
        [c.tool_name for c in self.tool_configs], excluded_colors=["green"]
    )
    # We now enter the router loop (until it returns something).
    while True:
        # Call the router to see what to do.
        output = self.router.route(chained_input.input)
        # Add the log to the Chained Input.
        chained_input.add(output.log, color="green")
        # If the tool chosen is the finishing tool, then we end and return.
        if output.tool == self.router.finish_tool_name:
            return {self.output_key: output.tool_input}
        # Otherwise we lookup the tool
        chain = name_to_tool_map[output.tool]
        # We then call the tool on the tool input to get an observation
        observation = chain(output.tool_input)
        # We then log the observation
        chained_input.add(f"\n{self.router.observation_prefix}")
        chained_input.add(observation, color=color_mapping[output.tool])
        # We then add the router prefix into the prompt to get the router to start
        # thinking, and start the loop all over.
        chained_input.add(f"\n{self.router.router_prefix}")

```

Once we have the custom router written, it is pretty easy to construct the routing chain:

```python
tools: List[ToolConfig] = ...
router = CustomRouter(....)
routing_chain = RoutingChain(tools=tools, router=router, verbose=True)
```